In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from adapt.feature_based import TCA

In [2]:
# loading WESAD data
subject_ids = [f"S{i}" for i in range(2, 18) if i not in [1, 12]]  
dfs = []

for sid in subject_ids:
    df = pd.read_csv(f"Processed WESAD Data/Processed_{sid}_data_unnormalised_new.csv")
    dfs.append(df)

wesad_data = pd.concat(dfs, ignore_index=True)
wesad_data

,Participant ID,Filtered EDA Mean,Filtered EDA Standard Deviation,SCL Mean,SCL Standard Deviation,SCR Mean,SCR Standard Deviation,HRV (RMSSD),HRV (SDNN),HR (bpm),TEMP Mean,TEMP Standard Deviation,TEMP Slope,TEMP Range,Label
0,2,1.319926,0.158933,1.311394,0.097091,0.048811,0.055282,227.981315,172.109971,85.243129,35.810000,0.022730,-0.000183,0.12,0
1,2,1.148486,0.123709,1.154106,0.116166,0.023503,0.031501,212.441958,170.307538,79.455212,35.768000,0.049322,-0.000651,0.21,0
2,2,0.969361,0.161578,0.967142,0.149690,0.011604,0.015669,184.233470,152.013695,78.617389,35.719333,0.036600,-0.000422,0.15,0
3,2,0.780818,0.099444,0.784546,0.106375,0.004581,0.006427,159.404043,153.345582,78.572928,35.708333,0.023464,0.000215,0.09,0
4,2,0.656155,0.073471,0.654764,0.072738,0.004088,0.006014,125.189743,136.537417,74.573025,35.744167,0.038828,0.000520,0.17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,17,1.085934,0.014432,1.085905,0.004904,0.003564,0.010046,107.596426,98.159150,125.852124,32.333167,0.070699,0.000994,0.27,1
1100,17,1.083978,0.007285,1.084305,0.005482,0.001510,0.003488,118.036298,116.497528,119.685699,32.427667,0.063715,0.000900,0.24,1
1101,17,1.075400,0.008162,1.075364,0.006820,0.001835,0.004210,152.322806,126.263518,112.021025,32.530750,0.076981,0.001091,0.26,1
1102,17,1.065623,0.011943,1.064553,0.008547,0.003008,0.004770,160.252394,131.740600,108.012618,32.616167,0.055230,0.000719,0.22,1


In [3]:
# loading affective_data
subject_ids = [f"Drv_{i}" for i in range(1, 14) if i != 2]  
affective_data = {}

for sid in subject_ids:
    left = pd.read_csv(f"Processed AffectiveROAD Data/{sid}_left_data_unnormalised_.csv")
    right = pd.read_csv(f"Processed AffectiveROAD Data/{sid}_right_data_unnormalised_.csv")
    combined = pd.concat([left, right], ignore_index=True)
    affective_data[sid] = combined

affective_data['Drv_1']

,Participant ID,Wrist,Filtered EDA Mean,Filtered EDA Standard Deviation,SCL Mean,SCL Standard Deviation,SCR Mean,SCR Standard Deviation,HRV (RMSSD),HRV (SDNN),HR (bpm),TEMP Mean,TEMP Standard Deviation,TEMP Slope,TEMP Range,Stress Label
0,1,Left,5.219569,0.231495,5.218068,0.152462,0.044282,0.095538,186.782481,170.410122,71.521511,34.825917,0.017078,0.000161,0.08,0.0
1,1,Left,5.388524,0.236291,5.375866,0.160373,0.051729,0.095987,130.869533,93.075210,67.794466,34.830583,0.015906,0.000006,0.06,0.0
2,1,Left,5.279265,0.386605,5.255102,0.309213,0.058357,0.100687,181.555839,127.773168,71.092304,34.817583,0.025674,-0.000321,0.10,0.0
3,1,Left,4.801925,0.395216,4.806559,0.367729,0.027595,0.051054,141.964733,108.598740,70.477189,34.805083,0.019386,-0.000005,0.07,0.0
4,1,Left,4.394909,0.180730,4.397087,0.199027,0.010146,0.014696,84.467369,61.161047,68.788198,34.811667,0.021422,0.000250,0.07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,Right,2.808709,0.166746,2.809205,0.160343,0.008685,0.013871,158.567781,105.902209,90.995261,34.356333,0.139546,-0.001966,0.40,1.0
76,1,Right,3.631054,0.132800,3.600742,0.070711,0.049462,0.086135,21.949279,31.387660,77.002639,36.735333,0.072272,0.001008,0.24,1.0
77,1,Right,3.740914,0.307837,3.833648,0.423779,0.030531,0.076302,56.221057,58.151319,76.699080,36.795583,0.028731,0.000258,0.14,1.0
78,1,Right,4.914828,0.379760,4.912760,0.323356,0.031633,0.102330,24.838477,32.150903,75.729687,36.727250,0.030454,-0.000014,0.11,1.0


In [4]:
# defining features
feature_col = ['HR (bpm)', 'HRV (RMSSD)', 'HRV (SDNN)', 'Filtered EDA Mean', 'Filtered EDA Standard Deviation', 'SCL Mean', 'SCL Standard Deviation', 'SCR Mean', 'SCR Standard Deviation', 'TEMP Mean', 'TEMP Standard Deviation', 'TEMP Slope', 'TEMP Range']

# defining X and y
X_wesad = wesad_data[feature_col]
y_train = wesad_data['Label']
groups = wesad_data['Participant ID']

In [11]:
score = []

# normalising WESAD training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_wesad)

# defining nn
model = Sequential([
    Dense(64, input_shape=(X_train_scaled.shape[1],), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),

    Dense(32, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),

    Dense(16, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# setting callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# training of WESAD data
model.fit(X_train_scaled, y_train,
          epochs=50, batch_size=32, verbose=0,
          validation_split=0.1,
          callbacks=[early_stopping, reduce_lr])

# looping through each of the Affective participants
for sid, df in affective_data.items():
    X_test_scaled = scaler.transform(df[feature_col])
    y_test = df['Stress Label']

    # predicting probabailites
    y_pred_prob = model.predict(X_test_scaled, verbose=0).flatten()

    # finding best threshold
    thresholds = np.linspace(0.3, 0.7, 21)
    f1_scores = [f1_score(y_test, (y_pred_prob > thr).astype(int)) for thr in thresholds]
    best_thr = thresholds[np.argmax(f1_scores)]

    # applying the threshold
    y_pred = (y_pred_prob > best_thr).astype(int)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    score.append({
        "Participant": sid,
        "Accuracy": acc,
        "F1 Score": f1
    })

results_df = pd.DataFrame(score)
results_df

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


,Participant,Accuracy,F1 Score
0,Drv_1,0.625000,0.583333
1,Drv_3,0.762500,0.765432
2,Drv_4,0.595238,0.630435
3,Drv_5,0.652174,0.714286
4,Drv_6,0.654762,0.579710
5,Drv_7,0.666667,0.740741
6,Drv_8,0.511364,0.481928
7,Drv_9,0.539474,0.255319
8,Drv_10,0.670455,0.567164
9,Drv_11,0.869565,0.846154


In [12]:
print(f" Average Accuracy = {results_df["Accuracy"].mean()}")

 Average Accuracy = 0.6378617100227512


In [13]:
print(results_df["F1 Score"].mean())

0.6136010223692177


In [11]:
def coral(source, target):
    # computing covariance matrix of source and target
    d = source.shape[1]
    cov_source = np.cov(source, rowvar=False) + np.eye(d) * 1e-5
    cov_target = np.cov(target, rowvar=False) + np.eye(d) * 1e-5

    # computing whitening and coloring transforms
    U_s, S_s, _ = np.linalg.svd(cov_source)
    U_t, S_t, _ = np.linalg.svd(cov_target)

    # whitening source
    sqrt_inv_S = np.diag(1.0 / np.sqrt(S_s))
    whitening = U_s @ sqrt_inv_S @ U_s.T

    # coloring to match target
    sqrt_S = np.diag(np.sqrt(S_t))
    coloring = U_t @ sqrt_S @ U_t.T

    # aligning 
    source_coral = (source - source.mean(axis=0)) @ whitening @ coloring + target.mean(axis=0)

    return source_coral

In [15]:
coral_ffnn_scores = []

# normalising WESAD data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_wesad)

for sid, df in affective_data.items():
    X_test = df[feature_col]
    y_test = df['Stress Label']
    X_test_scaled = scaler.transform(X_test)

    # applying CORAL
    X_train_coral = coral(X_train_scaled, X_test_scaled)

    # scaling training data
    coral_scaler = StandardScaler()
    X_train_coral_scaled = coral_scaler.fit_transform(X_train_coral)
    X_test_scaled_coral = coral_scaler.transform(X_test_scaled)

    # defining nn
    model = Sequential([
        Dense(64, input_shape=(X_train_coral_scaled.shape[1],), kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(32, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(16, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # setting callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # training
    model.fit(X_train_coral_scaled, y_train,
              epochs=50, batch_size=32, verbose=0,
              validation_split=0.1,
              callbacks=[early_stopping, reduce_lr])

    # predicting
    y_pred_prob = model.predict(X_test_scaled_coral, verbose=0).flatten()

    # setting and applying thresholds
    thresholds = np.linspace(0.3, 0.7, 21)
    f1_scores = [f1_score(y_test, (y_pred_prob > thr).astype(int)) for thr in thresholds]
    best_thr = thresholds[np.argmax(f1_scores)]

    y_pred = (y_pred_prob > best_thr).astype(int)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    coral_ffnn_scores.append({
        "Participant": sid,
        "Accuracy": acc,
        "F1 Score": f1
    })

results_coral_ffnn_df = pd.DataFrame(coral_ffnn_scores)
results_coral_ffnn_df

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negati

,Participant,Accuracy,F1 Score
0,Drv_1,0.787500,0.773333
1,Drv_3,0.712500,0.708861
2,Drv_4,0.630952,0.597403
3,Drv_5,0.750000,0.708861
4,Drv_6,0.547619,0.525000
5,Drv_7,0.654762,0.632911
6,Drv_8,0.454545,0.478261
7,Drv_9,0.368421,0.368421
8,Drv_10,0.727273,0.675676
9,Drv_11,0.891304,0.871795


In [16]:
print(f"Average Accuracy = {results_coral_ffnn_df['Accuracy'].mean()}")

Average Accuracy = 0.6439381556658444


In [17]:
print(f"Average Accuracy = {results_coral_ffnn_df['F1 Score'].mean()}")

Average Accuracy = 0.6192101093163066


In [18]:
importance_ffnn_scores = []

# normalising WESAD data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_wesad)

for sid, df in affective_data.items():
    X_test_scaled = scaler.transform(df[feature_col])
    y_test = df['Stress Label']

    # combining source and target for domain classification
    X_combined = np.vstack([X_train_scaled, X_test_scaled])
    y_domain = np.hstack([np.zeros(len(X_train_scaled)), np.ones(len(X_test_scaled))])

    # training domain classifier
    domain_clf = LogisticRegression(max_iter=1000)
    domain_clf.fit(X_combined, y_domain)

    # calculating weights: w_i = p_i / (1 - p_i)
    p_target = domain_clf.predict_proba(X_train_scaled)[:, 1]
    epsilon = 1e-6
    sample_weights = np.clip(p_target / (1 - p_target + epsilon), 0, 10)

    # defining nn
    model = Sequential([
        Dense(64, input_shape=(X_train_scaled.shape[1],), kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(32, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(16, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # setting callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # training nn with weights
    model.fit(X_train_scaled, y_train,
              sample_weight=sample_weights,
              epochs=50, batch_size=32, verbose=0,
              validation_split=0.1,
              callbacks=[early_stopping, reduce_lr])

    # predicting on each participant
    y_pred_prob = model.predict(X_test_scaled, verbose=0).flatten()
    thresholds = np.linspace(0.3, 0.7, 21)
    f1_scores = [f1_score(y_test, (y_pred_prob > thr).astype(int)) for thr in thresholds]
    best_thr = thresholds[np.argmax(f1_scores)]
    y_pred = (y_pred_prob > best_thr).astype(int)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    importance_ffnn_scores.append({
        "Participant": sid,
        "Accuracy": acc,
        "F1 Score": f1
    })

results_importance_ffnn_df = pd.DataFrame(importance_ffnn_scores)
results_importance_ffnn_df

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negati

,Participant,Accuracy,F1 Score
0,Drv_1,0.637500,0.524590
1,Drv_3,0.787500,0.767123
2,Drv_4,0.714286,0.692308
3,Drv_5,0.739130,0.720930
4,Drv_6,0.654762,0.567164
5,Drv_7,0.702381,0.719101
6,Drv_8,0.511364,0.598131
7,Drv_9,0.539474,0.255319
8,Drv_10,0.613636,0.451613
9,Drv_11,0.739130,0.625000


In [19]:
print(f"Average Accuracy = {results_importance_ffnn_df['Accuracy'].mean()}")

Average Accuracy = 0.6405651810686135


In [20]:
print(f"Average Accuracy = {results_coral_ffnn_df['F1 Score'].mean()}")

Average Accuracy = 0.6192101093163066


In [23]:

# creating a function to define nn to avoid error
def create_ffnn(input_dim):
    model = Sequential([
        Dense(64, input_shape=(input_dim,), kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(32, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(16, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tca_ffnn_scores = []

# normalising WESAD data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_wesad)

for sid, df in affective_data.items():
    X_test_scaled = scaler.transform(df[feature_col])
    y_test = df['Stress Label']

    # applying TCA
    tca = TCA(estimator=None, mu=10, n_components=16, kernel="linear")
    tca.fit(X_train_scaled, y_train, Xt=X_test_scaled)
    
    # transforming source and target using TCA
    X_train_tca = tca.transform(X_train_scaled)
    X_test_tca = tca.transform(X_test_scaled)

    # scaling
    tca_scaler = StandardScaler()
    X_train_tca_scaled = tca_scaler.fit_transform(X_train_tca)
    X_test_tca_scaled = tca_scaler.transform(X_test_tca)

    # defining nn
    model = create_ffnn(X_train_tca_scaled.shape[1])

    # setting callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # train nn
    model.fit(X_train_tca_scaled, y_train,
              epochs=50, batch_size=32, verbose=0,
              validation_split=0.1,
              callbacks=[early_stopping, reduce_lr])

    # predicting
    y_pred_prob = model.predict(X_test_tca_scaled, verbose=0).flatten()

    # setting and applying best threshold
    thresholds = np.linspace(0.3, 0.7, 21)
    f1_scores = [f1_score(y_test, (y_pred_prob > thr).astype(int)) for thr in thresholds]
    best_thr = thresholds[np.argmax(f1_scores)]
    y_pred = (y_pred_prob > best_thr).astype(int)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    tca_ffnn_scores.append({
        "Participant": sid,
        "Accuracy": acc,
        "F1 Score": f1
    })

results_tca_ffnn_df = pd.DataFrame(tca_ffnn_scores)

Fit transform...
Fit Estimator...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Fit transform...
Fit Estimator...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Fit transform...
Fit Estimator...


...

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Fit transform...
Fit Estimator...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Fit transform...
Fit Estimator...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [24]:
results_tca_ffnn_df

,Participant,Accuracy,F1 Score
0,Drv_1,0.625000,0.583333
1,Drv_3,0.737500,0.727273
2,Drv_4,0.607143,0.629213
3,Drv_5,0.641304,0.717949
4,Drv_6,0.642857,0.558824
5,Drv_7,0.583333,0.660194
6,Drv_8,0.443182,0.473118
7,Drv_9,0.578947,0.360000
8,Drv_10,0.636364,0.515152
9,Drv_11,0.869565,0.846154


In [25]:
results_tca_ffnn_df["Accuracy"].mean()


0.6336075799677059

In [26]:
results_tca_ffnn_df["F1 Score"].mean()

0.606558353620772